## Library

In [39]:
import os
import cv2
import heapq
import torch
import math
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence

In [3]:
# Print the path map
def plot_res(maze: torch.tensor, path=None, filename='tmp.png') -> None:
    
    res = maze.clone()
    row = res.shape[0]
    col = res.shape[1]
    
    rgb = torch.empty(row, col, 3)
    
    for i in range(row):
        for  j in range(col):
            # print(i, j, res[i, j], type(res[i, j]))
            rgb[i, j] = torch.tensor([0, 0, 0])
            
            if res[i, j] == 1:
                rgb[i, j] = torch.tensor([0, 0, 0])
                continue
            
            if res[i, j] == 0:
                rgb[i, j] = torch.tensor([255, 255, 255])
                continue
            
            if res[i, j] == -1:
                rgb[i ,j] = torch.tensor([0, 0, 0])
 
    if path:
        for step in path:
            rgb[step[0], step[1]] += torch.tensor([50, 50, 0])
                
    image = np.array(rgb).astype(np.uint8)
    # image = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    cv2.imwrite(filename, image)
    
# Covert images to .gif
def convert_to_gif(image_filenames, output_filename, duration=500, loop=0):
    """
    Converts a list of image filenames to an animated GIF.

    Args:
        image_filenames: A list of strings representing image file paths.
        output_filename: The filename for the resulting GIF.
        duration: Duration (in milliseconds) to display each frame in the GIF.
        loop: Number of times the animation should loop (0 for infinite).
    """
  
    # Check if all files exist
    for filename in image_filenames:
        if not os.path.isfile(filename):
            raise ValueError(f"File not found: {filename}")

    # Load the first image to get its size for consistency
    first_image = Image.open(image_filenames[0])
    width, height = first_image.size

    frames = []
    for filename in image_filenames:
        image = Image.open(filename).convert('RGB')  # Ensure RGB mode
        frames.append(image.copy())  # Append a copy to avoid modifying original

    # Save as GIF
    frames[0].save(output_filename, format='GIF', save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=loop)

## A* Algorithm

In [4]:
# A* algorithm implementation
def a_star_search(maze: torch, start: tuple, goal: tuple) -> list[tuple]:
    rows, cols = len(maze), len(maze[0])
    open_set = []
    heapq.heappush(open_set, (0, start))
    
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    idx = 0
    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            # path.append(start)
            return path[::-1]
        
        for direction in [
                (-1,  1), ( 0,  1), ( 1,  1),
                (-1,  0),           ( 1,  0),
                (-1, -1), ( 0, -1), ( 1, -1)
            ]:
            neighbor = (current[0] + direction[0], current[1] + direction[1])
            
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and maze[neighbor[0]][neighbor[1]] == 0:
                tentative_g_score = g_score[current] + 1
                
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return None  # No path found

In [40]:
maze = torch.tensor([
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
    [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
    [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0]], dtype=torch.int16)

# '1': wall '0': path

inverted_array = np.array([[not element for element in row] for row in maze])
image_array = (inverted_array * 255).astype(np.uint8)
image = cv2.cvtColor(image_array, cv2.COLOR_GRAY2BGR)
# cv2.imshow("Black and White Image", image)
# cv2.waitKey(0)  # Wait for a key press to close the window
# cv2.destroyAllWindows()
cv2.imwrite("maze.png", image)
maze.shape

torch.Size([22, 32])

In [6]:
def generate_binary_random_maze(size: tuple, start: tuple, goal: tuple, probability=0.1) -> torch.tensor:
    """
    Generates a binary random matrix of the given size with the specified probability of 1s.

    Args:
        size: A tuple representing the desired size of the matrix (e.g., (rows, columns)).
        probability: The probability (between 0 and 1) of assigning 1 to each element.

    Returns:
        A PyTorch tensor representing the binary random matrix.
    """

    # Generate random values between 0 and 1 (inclusive)
    rand_matrix = torch.rand(size)
    # Apply Bernoulli distribution to get 0s or 1s based on probability
    maze = torch.bernoulli(rand_matrix, probability).to(torch.int16)
    
    # Set start and golal to 'path'
    maze[start[0], start[1]] = 0
    maze[goal[0], goal[1]] = 0
      
    return maze

In [63]:
image_files = []
size = (16, 16)
start = (0, 0)
goal = (15, 15)

maze = generate_binary_random_maze(size=size, start=start, goal=goal, probability=0.2)
print(maze.shape)

path = a_star_search(maze=maze, start=(0, 0), goal=(size[0]-1, size[1]-1))

# path = a_star_search(maze, (0, 0), (21, 31))

if path:
    plot_res(maze, path, "res.png")

print(image_files)
convert_to_gif(image_files, 'res.gif', duration=100, loop=0)

torch.Size([16, 16])
['plot\\1.png', 'plot\\2.png', 'plot\\3.png', 'plot\\4.png', 'plot\\5.png', 'plot\\6.png', 'plot\\7.png', 'plot\\8.png', 'plot\\9.png', 'plot\\10.png', 'plot\\11.png', 'plot\\12.png', 'plot\\13.png', 'plot\\14.png', 'plot\\15.png', 'plot\\16.png', 'plot\\17.png', 'plot\\18.png', 'plot\\19.png']


In [7]:
def visualize(loc: tuple, goal: tuple, mist: torch.tensor, maze: torch.tensor, radius: int) -> torch.tensor:
    x_range = (max(0, loc[0]-radius), min(mist.shape[0], loc[0]+radius+1))
    y_range = (max(0, loc[1]-radius), min(mist.shape[1], loc[1]+radius+1))
    
    for i in range(x_range[0], x_range[1]):
        for j in range(y_range[0], y_range[1]):
            # Calculate the math distance
            if math.dist((i, j), loc) <= radius:
                mist[i, j] = maze[i, j]

                if (i, j) == goal:
                    return None
    return mist

def destination(loc: tuple, goal: tuple, mist: torch.tensor, maze: torch.tensor, radius: int) -> list[torch.tensor, list[tuple]]:
    dest = loc
    
    x_range = (max(0, loc[0]-radius), min(mist.shape[0], loc[0]+radius+1))
    y_range = (max(0, loc[1]-radius), min(mist.shape[1], loc[1]+radius+1))
    
    for i in range(x_range[0], x_range[1]):
        for j in range(y_range[0], y_range[1]):
            # Calculate the math distance
            if math.dist((i, j), loc) <= radius:
                mist[i, j] = maze[i, j]
                
                if (i, j) == goal:
                    dest = goal
    
    if dest != loc:
        return [mist, a_star_search(maze=mist, start=loc, goal=dest)]
    else:
        print('TypeError')
        return None      

In [8]:
def direct(loc: tuple, mist: torch.tensor, radius: int) -> tuple:
    
    current = loc
    maxBlock = 0
    move_dir = (0, 0)
    
    for direction in [
                (-1,  1), ( 0,  1), ( 1,  1),
                (-1,  0),           ( 1,  0),
                (-1, -1), ( 0, -1), ( 1, -1)
            ]:
        
        # Calculate the number of the black block to be discovered
        neighbor = (current[0] + direction[0], current[1] + direction[1])
        if neighbor[0] < 0 or neighbor[0] >= mist.shape[0] or neighbor[1] < 0 or neighbor[1] >= mist.shape[1] or mist[neighbor[0], neighbor[1]] != 0: continue
        
        x_range = (max(0, neighbor[0]-radius), min(mist.shape[0], neighbor[0]+radius+1))
        y_range = (max(0, neighbor[1]-radius), min(mist.shape[1], neighbor[1]+radius+1))
        
        block = 0
        for i in range(x_range[0], x_range[1]):
            for j in range(y_range[0], y_range[1]):
                # Calculate the math distance
                if mist[i, j] == -1 and math.dist((i, j), neighbor) <= radius:
                    block += 1
        
        if block > maxBlock:
            maxBlock = block
            move_dir = direction
    
    return move_dir

In [10]:
def nearest_bblock_path(start: tuple, mist: torch.tensor) -> list[tuple]:
    rows, cols = maze.shape[0], maze.shape[1]
    
    block_set = []
    
    for i in range(rows):
        for j in range(cols):
            if mist[i, j] == -1:
                heapq.heappush(block_set, (math.dist((i, j), start), (i, j)))
    
    while block_set:
        _, block_loc = heapq.heappop(block_set)
        
        Apath = block_search(maze=mist, start=start, goal=block_loc)
        
        if Apath: return Apath
    
    return None
        
# Define the heuristic function
def heuristic(a: tuple, b: tuple) -> float:
    return math.dist(a, b)  # Return the distance

# A* algorithm implementation
def block_search(maze: torch, start: tuple, goal: tuple) -> list[tuple]:
    rows, cols = len(maze), len(maze[0])
    open_set = []
    heapq.heappush(open_set, (0, start))
    
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        for direction in [
                (-1,  1), ( 0,  1), ( 1,  1),
                (-1,  0),           ( 1,  0),
                (-1, -1), ( 0, -1), ( 1, -1)
            ]:
            neighbor = (current[0] + direction[0], current[1] + direction[1])
            
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and maze[neighbor[0]][neighbor[1]] != 1:
                tentative_g_score = g_score[current] + 1
                
                if maze[neighbor[0], neighbor[1]] == -1:
                    path = []
                    while current in came_from:
                        path.append(current)
                        current = came_from[current]
                    path.append(start)
                    return path[::-1]
                
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return None  # No path found

In [14]:
image_files = []

radius = 3
size = (128, 128)
start = (0, 0)
goal = (24, 0)

maze = generate_binary_random_maze(size=size, start=start, goal=goal, probability=0.18)

inverted_array = np.array([[not element for element in row] for row in maze])
image_array = (inverted_array * 255).astype(np.uint8)
image = cv2.cvtColor(image_array, cv2.COLOR_GRAY2BGR)
cv2.imwrite("maze.png", image)

True

In [15]:
mist = torch.bernoulli(torch.rand(size=size), p=1).to(torch.int16) * (-1)
print(maze.shape)

current = start
path = [current]
Apath = []
Gpath = []

idx = 0
while current != goal:
    idx += 1
    plot_res(mist, path, f"tmp\\{idx}.png")
    image_files.append(f"tmp\\{idx}.png")
    
    vision = visualize(loc=current, goal=goal, mist=mist, maze=maze, radius=radius)
    if vision is not None:
        mist = vision
    else:
        mist,  Gpath = destination(loc=current, goal=goal, mist=mist, maze=maze, radius=radius)
    
    direction = direct(loc=current, mist=mist, radius=radius)
    
    # print(direction)
    if direction != (0, 0):
        current = (current[0] + direction[0], current[1] + direction[1])
        path.append(current)
    else:
        Apath = nearest_bblock_path(start=current, mist=mist)
        for current in Apath:
            idx += 1
            plot_res(mist, path, f"tmp\\{idx}.png")
            image_files.append(f"tmp\\{idx}.png")
            path.append(current)
            
            vision = visualize(loc=current, goal=goal, mist=mist, maze=maze, radius=radius)
            if vision is not None:
                mist = vision
            else:
                mist,  Gpath = destination(loc=current, goal=goal, mist=mist, maze=maze, radius=radius)
                for current in Gpath:
                    idx += 1
                    plot_res(mist, path, f"tmp\\{idx}.png")
                    image_files.append(f"tmp\\{idx}.png")
                    path.append(current)
                break
            

print(image_files)
convert_to_gif(image_files, 'tmp.gif', duration=80, loop=0)

torch.Size([128, 128])
['tmp\\1.png', 'tmp\\2.png', 'tmp\\3.png', 'tmp\\4.png', 'tmp\\5.png', 'tmp\\6.png', 'tmp\\7.png', 'tmp\\8.png', 'tmp\\9.png', 'tmp\\10.png', 'tmp\\11.png', 'tmp\\12.png', 'tmp\\13.png', 'tmp\\14.png', 'tmp\\15.png', 'tmp\\16.png', 'tmp\\17.png', 'tmp\\18.png', 'tmp\\19.png', 'tmp\\20.png', 'tmp\\21.png', 'tmp\\22.png', 'tmp\\23.png', 'tmp\\24.png', 'tmp\\25.png', 'tmp\\26.png', 'tmp\\27.png', 'tmp\\28.png', 'tmp\\29.png', 'tmp\\30.png', 'tmp\\31.png', 'tmp\\32.png', 'tmp\\33.png', 'tmp\\34.png', 'tmp\\35.png', 'tmp\\36.png', 'tmp\\37.png', 'tmp\\38.png', 'tmp\\39.png', 'tmp\\40.png', 'tmp\\41.png', 'tmp\\42.png', 'tmp\\43.png', 'tmp\\44.png', 'tmp\\45.png', 'tmp\\46.png', 'tmp\\47.png', 'tmp\\48.png', 'tmp\\49.png', 'tmp\\50.png', 'tmp\\51.png', 'tmp\\52.png', 'tmp\\53.png', 'tmp\\54.png', 'tmp\\55.png', 'tmp\\56.png', 'tmp\\57.png', 'tmp\\58.png', 'tmp\\59.png', 'tmp\\60.png', 'tmp\\61.png', 'tmp\\62.png', 'tmp\\63.png', 'tmp\\64.png', 'tmp\\65.png', 'tmp\\66.p

In [44]:
test = None
assert test is not None, 'hello'

AssertionError: hello